In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import statsmodels.formula.api as smf
from glob import glob
from itertools import combinations
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import median_absolute_error
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import f_regression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.feature_selection import f_regression
from sklearn.model_selection import cross_val_score

# Proyecto 1. Aprendizaje de máquina probabilístico

El objetivo del proyecto es predecir el número de victorias de la temporada siguiente de cada equipo utilizando información de la temporada pasada. Por lo tanto la variable objetivo, $y:=\text{Número de victorias de la temporada.}$

Las temporadas condideradas fueron de los años 2004 a 2020 y puesto que hay 32 equipos hubo un total de 544 temporadas consideradas. Como son tan solo 544 temporadas no dividimos la muestra en entrenamiento y prueba. Por lo tanto, el mejor modelo se decidirá utilizando k-fold cross validation para encontrar aquel que minimice el mean absolute error promedio.

El error absoluto medio (MAE) se calcula de la siguiente manera:

$$MAE=\sum_{i=1}^{n} \frac{|y_{i}-\hat{y}_{i}|}{n}$$

Donde:

- $y_{i}$ es el número de victorias en la temporada.
- $\hat{y}_{i}$ es el número de victorias en la temporada predecida por la regresión.
- $n$ es el número de observaciones.

In [2]:
# Leemos los datos
datos_nfl=pd.read_csv(r"/Users/damian/Documents/Maestria 2025-01/Aprendizaje maquina probabilistico/Proyecto intermedio/Datos/team_stats_2003_2023.csv").rename(columns={'pass_yds':'net_pass_yds'})
# Quitamos los datos de 2021 a 2023
datos_nfl=datos_nfl[datos_nfl["year"]<=2020].reset_index(drop=True)
# Llenamos los missing 
datos_nfl["ties"]=datos_nfl["ties"].fillna(0)
# Creamos una columna con las victorias y empates tomando los empates como 0.5
datos_nfl["wins_ties"]=datos_nfl["wins"]+0.5*datos_nfl["ties"]
# Creamos la columna de win loss percentage
datos_nfl["win_loss_perc"]=datos_nfl["wins_ties"]/datos_nfl["g"]
# Pythagorean wins (victorias esperadas)
datos_nfl["pythagoran_wins"]=(datos_nfl["points"]**(2.37)/(datos_nfl["points"]**(2.37)+datos_nfl["points_opp"]**(2.37)))*datos_nfl["g"]
# Pythagorean win percentage
#datos_nfl["pythagoran_win_percentage"]=(datos_nfl["points"]**(2.37)/(datos_nfl["points"]**(2.37)+datos_nfl["points_opp"]**(2.37)))
# Calculamos la diferencia entre win percentage y el pythagorean win percentage
#datos_nfl["Difference_pythagoran_win"]=datos_nfl["win_loss_perc"]-datos_nfl["pythagoran_win_percentage"]

In [3]:
def handle_nfl_passer_rating(cmpls, yds, tds, ints):
     """ Defines a function which handles passer rating calculation for the NFL."""
     def _min_max(x, xmin, xmax):
         """
         Defines a function which enforces a minimum and maximum value.
         Input: x, the value to check.
         Input: xmin, the minumum value.
         Input: xmax, the maximum value.
         Output: Either x, xmin or xmax, depending.
         """
         # Check if x is less than the minimum. If so, return the minimum.
         if x < xmin:
             return xmin
         
         # Check if x is greater than the maximum. If so, return the maximum.
         elif x > xmax:
             return xmax
         
         # Otherwise, just return x. And weep for the future.
         else:
             return x
             
     # Step 0: Make sure these are floats, dammit.
     cmpls = cmpls + 0.0
     yds = yds + 0.0
     tds = tds + 0.0
     ints = ints + 0.0
     
     # Step 1: The completion percentage.         
     step_1 = cmpls - 0.3
     step_1 = step_1 * 5
     step_1 = _min_max(step_1, 0, 2.375)
     
     # Step 2: The yards per attempt.
     step_2 = yds - 3
     step_2 = step_2 * 0.25
     step_2 = _min_max(step_2, 0, 2.375)
     
     # Step 3: Touchdown percentage.
     step_3 = tds * 20
     step_3 = _min_max(step_3, 0, 2.375)
     
     # Step 4: Interception percentage.
     step_4 = ints * 25
     step_4 = 2.375 - step_4
     step_4 = _min_max(step_4, 0, 2.375)
     
     # Step 5: Compute the rating based on the sum of steps 1-4.
     rating = step_1 + step_2 + step_3 + step_4 + 0.0
     rating = rating / 6
     rating = rating * 100
     
     # Step 6: Return the rating, formatted to 1 decimal place, as a Decimal.
     return rating

In [4]:
# Funcion para renombrar a los redskins a Football Team
def fun_cam_nom(team_name):
    if team_name=='Washington Redskins':
        new_team_name='Washington Football Team'
    elif team_name=='San Diego Chargers':
        new_team_name='Los Angeles Chargers'
    elif team_name=='St. Louis Rams':
        new_team_name='Los Angeles Rams'
    elif team_name=='Oakland Raiders':
        new_team_name='Las Vegas Raiders'
    else:
        new_team_name=team_name
    return new_team_name

# Renombramos el nombre de redskins
datos_nfl["team"]=datos_nfl.apply(lambda x: fun_cam_nom(x.team), axis=1)

## Homologamos las claves de los equipos

In [5]:
# Leemos la base de pro-football reference
df_pro_football_reference=pd.read_csv(r"/Users/damian/Documents/Maestria 2025-01/Aprendizaje maquina probabilistico/Proyecto intermedio/Datos/Seasons/Season-2020.csv")

In [6]:
# Para agregar los nombres
df_tm_nano=df_pro_football_reference.drop_duplicates(subset=["tm_nano"]).sort_values(by=["tm_name"])
df_tm_nano['team']=df_tm_nano["tm_market"]+' '+df_tm_nano["tm_name"]

In [7]:
# Quitamos columnas para hacer el merge
df_tm_nano_slim=df_tm_nano[["tm_nano","team"]].reset_index(drop=True)

In [8]:
# Agregamos el tm_nano en la base original
datos_nfl_tn=datos_nfl.merge(df_tm_nano_slim, how='left', on="team")

In [9]:
# Para hacer los desempates
# Creamos una variables uniformes para hacer el desempate
rng = np.random.default_rng(seed=69)
b=rng.uniform(size=10)

lista=[]
for i in b:
    if i>0.5:
        lista=lista+[1,0]
    else:
        lista=lista+[0,1]
# Lista para agregar victorias
lista_array=np.array(lista)


In [10]:
# Agregamos una columna de puros ceros 
datos_nfl_tn["extra_win"]=0
datos_nfl_tn["extra_loss"]=0
j=0
# Agregamos la columna con la victoria extra
for i in datos_nfl_tn.index.values:
    if datos_nfl_tn.loc[i, "ties"]==1:
        datos_nfl_tn.loc[i, "extra_win"]=lista_array[j]
        datos_nfl_tn.loc[i, "extra_loss"]=abs(lista_array[j]-1)
        j=j+1
    else:
        datos_nfl_tn.loc[i, "extra_win"]=0
        datos_nfl_tn.loc[i, "extra_loss"]=0
    
# Creamos la variable de victorias to
datos_nfl_tn["total_wins"]=datos_nfl_tn["wins"]+datos_nfl_tn["extra_win"]
datos_nfl_tn["total_losses"]=datos_nfl_tn["losses"]+datos_nfl_tn["extra_loss"]
        

## Medimos la autocorrelación de las series de tiempo de victorias

Utilizamos la prueba de Ljung–Box para determinar si hay correlación entre las observaciones con un lag máximo de 3. La hipótesis nula es que no hay autocorrelación entre las observaciones, es decir que los datos son independientes. Mientras que la hipótesis alternativa es que los datos no son independientes.

In [11]:
def test_auto(df,team, lags=3):
    # Print PACF values
    #fit ARMA model to dataset
    pru=df[df["team"]==team][["year","wins"]]
    pru.set_index('year', inplace=True)

    # Calulamos el test Ljung–Box para el lag hasta 3
    df_lb=sm.stats.diagnostic.acorr_ljungbox(pru.values, lags=lags)
    df_lb.insert(0, "Team", team)
    df_lb.insert(3, "Lag", [1,2,3])
    return df_lb

In [12]:
# Lo corremos para todos los equipos
df_lb_c=pd.DataFrame()
for j, team in enumerate(datos_nfl_tn["team"].unique()):
    df_lb=test_auto(datos_nfl_tn,team)
    df_lb_c=pd.concat([df_lb_c,df_lb]).reset_index(drop=True)

# Veamos que valores quedan menor a 0.05
df_lb_c.sort_values(by=["lb_pvalue"]).reset_index(drop=True).head(7)

,Team,lb_stat,lb_pvalue,Lag
0,Denver Broncos,6.008775,0.014235,1
1,Seattle Seahawks,4.929742,0.026398,1
2,Miami Dolphins,4.433237,0.035246,1
3,Denver Broncos,6.166832,0.045803,2
4,Seattle Seahawks,6.088915,0.047622,2
5,Seattle Seahawks,7.452301,0.058797,3
6,New York Jets,5.663010,0.058924,2


In [13]:
# El porcentaje de casos que quedan menor a 0.05
por_men_05=len(df_lb_c[df_lb_c["lb_pvalue"]<0.05]["Team"].unique())/len(df_lb_c["Team"].unique())
por_men_05

0.09375

Como menos del 10% de los equipos mostraron algún tipo de autocorrelación con los lag de 1, 2 y 3. Entonces, podemos suponer que las observaciones de victorias por temporada son independientes para todos los equipos.

## Agregamos los datos por temporada más desglosados

In [14]:
# Abrimos los datos de las temporadas desplosados
# Get the absolute paths of all Excel files 
seasons_splits_dir = glob("/Users/damian/Documents/Maestria 2025-01/Aprendizaje maquina probabilistico/Proyecto intermedio/Datos/Season_splits/*.csv")

# Guardamos todo en un data frame
df_season_splits=pd.concat(pd.read_csv(csv_file) for csv_file in seasons_splits_dir)

# Nos quedamos solo con home y away
df_season_splits_r=df_season_splits[df_season_splits["splits_type"].isin(["Home","Away"])].sort_values(by=["season"]).reset_index(drop=True)

# Dividimos entre la ofensiva 
df_season_splits_off=df_season_splits_r[df_season_splits_r["splits_side"]=="For"].drop(columns=["market","alias","splits_by","splits_side"])
# y defensiva
df_season_splits_def=df_season_splits_r[df_season_splits_r["splits_side"]=="Against"].drop(columns=["market","alias","splits_by","splits_side"])

In [15]:
# Calculamos las métricas para la ofensiva
# Primero calculamos las yardas totales (sin contar capturas)
df_season_passing=df_season_splits_off.groupby(by=['season','nano','name']).agg({'pass_yds':'sum','pass_cmp':'sum','pass_att':'sum','pass_tds':'sum','pass_int':'sum'}).reset_index()
# Calculamos el passer rating
df_season_passing["qb_rating"]=df_season_passing.apply(lambda x: handle_nfl_passer_rating(x.pass_cmp/x.pass_att, x.pass_yds/x.pass_att, x.pass_tds/x.pass_att, x.pass_int/x.pass_att), axis=1)
# Yardas por pase
df_season_passing["yards_per_attempt"]=df_season_passing["pass_yds"]/df_season_passing["pass_att"]
# Base final solo con las variables necesarias
df_season_passing_f=df_season_passing[["season","nano","pass_yds","qb_rating","yards_per_attempt"]]

In [16]:
# Calculamos las métricas de la defensiva
df_season_def=df_season_splits_def.groupby(by=['season','nano','name']).agg({'pass_yds':'sum','pass_cmp':'sum','pass_att':'sum','pass_tds':'sum','pass_int':'sum'}).reset_index()

# Calculamos el passer rating
df_season_def["opp_qb_rating"]=df_season_def.apply(lambda x: handle_nfl_passer_rating(x.pass_cmp/x.pass_att, x.pass_yds/x.pass_att, x.pass_tds/x.pass_att, x.pass_int/x.pass_att), axis=1)
# Nada más nos quedamos con algunas variables
df_season_def_f=df_season_def[["season","nano","opp_qb_rating"]]

In [17]:
# Juntamos las bases
df_split_agg_c=df_season_passing_f.merge(df_season_def_f, on=["season","nano"], how='inner')

# Agregamos el nombre 

df_split_agg_cf=df_split_agg_c.merge(df_season_splits_def.drop_duplicates(subset=["name"])[["nano","name"]], on='nano',how='left')

## Datos extra para los fumbles

In [18]:
# Leemos los datos
df_datos_extra=pd.read_csv("/Users/damian/Documents/Maestria 2025-01/Aprendizaje maquina probabilistico/Proyecto intermedio/Datos/Datos_extra/nfl_team_stats_2002-2023.csv")
# Quitamos algunas cosas
df_datos_extra_r=df_datos_extra[(~df_datos_extra["week"].isin(['Wildcard', 'Division', 'Conference','Superbowl'])) & (df_datos_extra["season"]<=2020) & (df_datos_extra["season"]>=2003) ]

In [19]:
# Hacemos la agrupación de seasons
# Primero guardamos los id distintos
id_dist_extra=df_datos_extra_r["home"].unique()

df_info_extra_agg=pd.DataFrame()

for id_t in id_dist_extra:
    # Prueba para agrupar por equipo
    prueba=df_datos_extra_r[(df_datos_extra_r["away"]==id_t) | (df_datos_extra_r["home"]==id_t)].reset_index(drop=True)
    # Agregemos la variable que identifica
    prueba["name"]=id_t
    # Renombramos a los commies
    prueba["name"]=np.where(prueba["name"]=="Commanders","Football Team",prueba["name"])
    # Fumbles home
    prueba["fum_def_h"]=np.where(prueba["home"]==id_t, prueba["fumbles_away"], 0)
    # Fumbles away
    prueba["fum_def_a"]=np.where(prueba["away"]==id_t, prueba["fumbles_home"], 0)
    # Intercepciones home
    prueba["int_def_h"]=np.where(prueba["home"]==id_t, prueba["interceptions_away"], 0)
    # Intercepciones away
    prueba["int_def_a"]=np.where(prueba["away"]==id_t, prueba["interceptions_home"], 0)
    # Intercepciones totales
    prueba["pass_int_def"]=prueba["int_def_h"]+prueba["int_def_a"]
    # Fumbles totales
    prueba["fum_def"]=prueba["fum_def_h"]+prueba["fum_def_a"]
    # Agrupamos
    prueba_agrupada_extra=prueba.groupby(by=["name","season"]).agg({'fum_def':'sum','pass_int_def':'sum'}).reset_index()
    df_info_extra_agg=pd.concat([df_info_extra_agg,prueba_agrupada_extra])

In [20]:
# Juntamos las bases
df_split_complete=df_split_agg_cf.merge(df_info_extra_agg, on=['season','name']).drop(columns='name').rename(columns={'nano':'tm_nano','season':'year'})
# Todos los balones perdidos que forzó la defensiva
df_split_complete["forzed_turnovers_def"]=df_split_complete["fum_def"]+df_split_complete["pass_int_def"]

## Agregamos los datos de seasons

In [21]:
# Abrimos los datos de las temporadas
# Get the absolute paths of all Excel files 
seasons_dir = glob("/Users/damian/Documents/Maestria 2025-01/Aprendizaje maquina probabilistico/Proyecto intermedio/Datos/Seasons/*.csv")

df_seasons=pd.concat(pd.read_csv(csv_file) for csv_file in seasons_dir)

# Quitamos las semanas de postemporada
df_seasons=df_seasons[df_seasons["week"]<=17].reset_index(drop=True).sort_values(by=["season","week"])

In [22]:
# Hacemos la agrupación de seasons
# Primero guardamos los id distintos
id_dist=df_seasons["tm_nano"].unique()

df_seasons_agg=pd.DataFrame()

for id_t in id_dist:
    # Prueba para agrupar por equipo
    prueba=df_seasons[(df_seasons["tm_nano"]==id_t) | (df_seasons["opp_nano"]==id_t)].reset_index(drop=True)
    # Agregemos la variable que identifica
    prueba["id"]=id_t
    # Margin of victory
    prueba["margin_victory"]=np.where(prueba["tm_nano"]==id_t, prueba["tm_score"]-prueba["opp_score"], prueba["opp_score"]-prueba["tm_score"])
    # One score-game
    prueba["one_score_game"]=np.where(abs(prueba["tm_score"]-prueba["opp_score"])<=8, 1, 0)
    # One score-wins
    prueba["one_score_win"]=np.where(np.logical_and(prueba["one_score_game"]==1, prueba["margin_victory"]>0), 1, 0)
    # Agrupamos
    prueba_agrupada=prueba.groupby(by=["id","season"]).agg({'margin_victory':'mean', 'one_score_game':'sum', 'one_score_win':'sum'}).reset_index()
    df_seasons_agg=pd.concat([df_seasons_agg,prueba_agrupada])

# Renombramos las variables
df_seasons_agg=df_seasons_agg.rename(columns={'id':'tm_nano','season':'year'})

In [23]:
# Juntamos todas las bases con los datos
df_seasons_agg_f=df_seasons_agg.merge(df_split_complete, on=['tm_nano','year'])

In [24]:
# Juntamo las bases
datos_nfl_cs=datos_nfl_tn.merge(df_seasons_agg_f, how='left', on=["tm_nano",'year'])
# Creamos variables
# Diferencial de balones sueltos
datos_nfl_cs["turnover_diff"]=datos_nfl_cs["forzed_turnovers_def"]-datos_nfl_cs["turnovers"]
# El ranking de turnover differential por año
datos_nfl_cs["turnover_diff_rank_per_y"]=datos_nfl_cs.groupby(['year'])['turnover_diff'].rank(ascending=False, method='average')
# Turnover differential mayor a 15
datos_nfl_cs["turnover_diff_rank_per_y_m_15"]=np.where(datos_nfl_cs["turnover_diff_rank_per_y"]>=15,1,0)
# Turnover differential menor a 15
#datos_nfl_cs["turnover_diff_rank_per_y_men_15"]=np.where(datos_nfl_cs["turnover_diff_rank_per_y"]<=-15,1,0)
# Porcentaje de victoria en one-score games
datos_nfl_cs["win_per_in_one_score"]=datos_nfl_cs["one_score_win"]/datos_nfl_cs["one_score_game"]
# Porcentaje de victoria en one-score games
datos_nfl_cs["diff_one_score_wins_games"]=datos_nfl_cs["one_score_win"]-datos_nfl_cs["one_score_game"]
# Porcentaje de victoria en one-score games
datos_nfl_cs["diff_one_score_wins_games_m_5"]=np.where(datos_nfl_cs["diff_one_score_wins_games"]<=-5,1,0)
# Porcentaje de victoria en one-score games
datos_nfl_cs["diff_one_score_wins_games_m_4"]=np.where(datos_nfl_cs["diff_one_score_wins_games"]<=-4,1,0)
# Porcentaje de partidos que se ganaron con one score
datos_nfl_cs["prop_one_score_wins"]=np.where(datos_nfl_cs["total_wins"]==0,0,datos_nfl_cs["one_score_win"]/datos_nfl_cs["total_wins"])
# Tiene más 6 o más one-score-games
datos_nfl_cs["five_more_one_score"]=np.where(datos_nfl_cs["one_score_game"]>=6, 1,0)
# Tiene más 6 o más one-score-games
datos_nfl_cs["six_more_one_score"]=np.where(datos_nfl_cs["one_score_game"]>=6, 1,0)
# Tiene un porcentaje de 0.7 o mayor de victorias en one-score games
datos_nfl_cs["equal_more_per_70_wins_one_score"]=np.where(np.logical_and(datos_nfl_cs["five_more_one_score"]==1,datos_nfl_cs["win_per_in_one_score"]>0.7),1,0)
# Tiene un porcentaje de 0.7 o mayor de victorias en one-score games
datos_nfl_cs["equal_more_per_70_wins_one_score_6"]=np.where(np.logical_and(datos_nfl_cs["six_more_one_score"]==1,datos_nfl_cs["win_per_in_one_score"]>0.7),1,0)
# Tiene un porcentaje de 0.7 o mayor de victorias en one-score games
datos_nfl_cs["equal_more_per_80_wins_one_score"]=np.where(np.logical_and(datos_nfl_cs["five_more_one_score"]==1,datos_nfl_cs["win_per_in_one_score"]>0.8),1,0)
# Tiene un porcentaje de 0.7 o mayor de victorias en one-score games
datos_nfl_cs["equal_more_per_80_wins_one_score_6"]=np.where(np.logical_and(datos_nfl_cs["six_more_one_score"]==1,datos_nfl_cs["win_per_in_one_score"]>0.8),1,0)
# Puntos por partido
datos_nfl_cs["points_per_game"]=datos_nfl_cs["points"]/datos_nfl_cs["g"]
# Puntos por partido del oponent
datos_nfl_cs["opp_points_per_game"]=datos_nfl_cs["points_opp"]/datos_nfl_cs["g"]
# Yardas de pase por partido
datos_nfl_cs["pass_yards_game"]=datos_nfl_cs["net_pass_yds"]/datos_nfl_cs["g"]
# Yardas corridas por partido
datos_nfl_cs["rush_yards_game"]=datos_nfl_cs["rush_yds"]/datos_nfl_cs["g"]
# Castigos por partido
datos_nfl_cs["pentalies_game"]=datos_nfl_cs["penalties"]/datos_nfl_cs["g"]
# Calculamos la diferencia entre win y el pythagorean win
datos_nfl_cs["Difference_pythagoran_win"]=datos_nfl["wins_ties"]-datos_nfl["pythagoran_wins"]
# Porcetaje de primeros y dieces por castigo
datos_nfl_cs["pen_fd_per"]=datos_nfl_cs["pen_fd"]/(datos_nfl_cs["pass_fd"]+datos_nfl_cs["rush_fd"]+datos_nfl_cs["pen_fd"])
# Porcentaje de primeros y dieces por pase
datos_nfl_cs["pass_fd_per"]=datos_nfl_cs["pass_fd"]/(datos_nfl_cs["pass_fd"]+datos_nfl_cs["rush_fd"]+datos_nfl_cs["pen_fd"])
# Porcentaje de primeros y dieces por corrida
datos_nfl_cs["rush_fd_per"]=datos_nfl_cs["rush_fd"]/(datos_nfl_cs["pass_fd"]+datos_nfl_cs["rush_fd"]+datos_nfl_cs["pen_fd"])
# Porcentaje de pases completos
datos_nfl_cs["pass_cmp_per"]=datos_nfl_cs["pass_cmp"]/datos_nfl_cs["pass_att"]
# Porcentaje de touchdows por pase
datos_nfl_cs["pass_td_per"]=datos_nfl_cs["pass_td"]/datos_nfl_cs["pass_att"]
# Porcentaje de intercepciones por pase
datos_nfl_cs["pass_int_per"]=datos_nfl_cs["pass_int"]/datos_nfl_cs["pass_att"]
# Proporción entre pythagorean wins y victorias
datos_nfl_cs["prop_pyth_win_a_win"]=datos_nfl_cs["total_wins"]/datos_nfl_cs["pythagoran_wins"]
# Proporción entre pythagorean wins y victorias menor a 0.76
datos_nfl_cs["prop_pyth_win_a_win_me_7"]=np.where(datos_nfl_cs["prop_pyth_win_a_win"]<=0.76,1,0)
# Proporción entre pythagorean wins y victorias mayor a 1.20
datos_nfl_cs["prop_pyth_win_a_win_ma_12"]=np.where(datos_nfl_cs["prop_pyth_win_a_win"]>=1.2,1,0)
# Margin of victory menor a 1
datos_nfl_cs["margin_victory_m_1"]=np.where(np.abs(datos_nfl_cs["margin_victory"])<=1,1,0)

# Agregamos el qb rating promedio
datos_nfl_cs_f=datos_nfl_cs.merge(datos_nfl_cs.groupby(['year']).agg({'qb_rating':'mean'}).reset_index().rename(columns={'qb_rating':'qb_rating_avg_y'}), on='year', how='left')

# Calculamos la diferencia entre el qb rating de la ofensiva y el promedio anual
datos_nfl_cs_f["diff_qb_rating_offense"]=datos_nfl_cs_f["qb_rating"]-datos_nfl_cs_f["qb_rating_avg_y"]
# Calculamos la diferencia entre el qb rating que la defensiva enfrentó y el promedio anual
datos_nfl_cs_f["diff_qb_rating_defense"]=datos_nfl_cs_f["opp_qb_rating"]-datos_nfl_cs_f["qb_rating_avg_y"]

# Pythagorean wins (victorias esperadas)
datos_nfl_cs_f["diff_pythagoran_wins"]=datos_nfl_cs_f["total_wins"]-datos_nfl_cs_f["pythagoran_wins"]

# Quitamos el qb_promedio
datos_nfl_cs_f=datos_nfl_cs_f.drop(columns=["qb_rating_avg_y"])

# Ranking de ofensiva total
#datos_nfl_cs["rank_total_offense"]=np.where(datos_nfl_cs["prop_pyth_win_a_win"]>=1.2,1,0)

# Quitamos el win_loss_perc, total_losses
datos_nfl_cs_f=datos_nfl_cs_f.drop(columns=["win_loss_perc","wins_ties","ties","wins_ties","mov","pass_int","pass_int_def","extra_loss"])

In [25]:
# Primero guardamos los id distintos
id_dist=datos_nfl_cs_f["tm_nano"].unique()

df_seasons_cf=pd.DataFrame()

for id_t in id_dist:
    # Recorremos los datos un renglón
    prueba_2=datos_nfl_cs[datos_nfl_cs["tm_nano"]==id_t].sort_values(by=["team", "year"]).copy()
    prueba_2.insert(0,'id',prueba_2["tm_nano"])

    # Seleccionamos el equipo
    prueba_2_c=prueba_2[prueba_2["id"]==id_t]

    # Recorremos los datos un renglón
    prueba_2_c.iloc[:,6:]=prueba_2_c.iloc[:,6:].shift(1)

    # Le agregamos el suffix para saber que es el del año anterior
    prueba_2_c=prueba_2_c.add_suffix('_y_1')

    # Renombramos la columnas
    prueba_2_c=prueba_2_c.rename(columns={'id_y_1':'id', 'year_y_1': 'year', 'team_y_1':'team'})

    # Quitamos el año 2003 porque para ese no tenemos datos
    prueba_2_cf=prueba_2_c[prueba_2_c["year"]!=2003]
    # Quitamos columnas poco relevantes
    prueba_2_cf=prueba_2_cf.drop(columns=["wins_y_1","losses_y_1","win_loss_perc_y_1","mov_y_1","tm_nano_y_1","g_y_1","pass_int_y_1","pass_int_def_y_1","extra_win_y_1","ties_y_1","extra_loss_y_1","wins_ties_y_1"])
    

    # Guardamos en una tabla
    df_seasons_cf=pd.concat([df_seasons_cf,prueba_2_cf]).reset_index(drop=True)

In [26]:
# Tomamos todos los datos
df_seasons_ce=df_seasons_cf.sort_values(by=["year","team"]).reset_index(drop=True)

# Variable objetivo
# Base de entranamiento de solo las variables objetivo
datos_var_obj_e=datos_nfl_cs[datos_nfl_cs["year"]!=2003][["year", "team", "tm_nano","total_wins", "total_losses"]].sort_values(by=["year","team"]).reset_index(drop=True).rename(columns={'tm_nano':'id'})

# Juntamos todo en una base
df_seasons_total=df_seasons_ce.merge(datos_var_obj_e, on=["id","year","team"], how='inner')

In [27]:
df_seasons_total

,id,year,team,points_y_1,points_opp_y_1,points_diff_y_1,total_yards_y_1,plays_offense_y_1,yds_per_play_offense_y_1,turnovers_y_1,...,rush_fd_per_y_1,pass_cmp_per_y_1,pass_td_per_y_1,pass_int_per_y_1,prop_pyth_win_a_win_y_1,prop_pyth_win_a_win_me_7_y_1,prop_pyth_win_a_win_ma_12_y_1,margin_victory_m_1_y_1,total_wins,total_losses
0,GJizlkzQFaqGA,2004,Arizona Cardinals,225.0,452.0,-227.0,4490.0,981.0,4.6,36.0,...,0.312500,0.567416,0.033708,0.041199,1.556007,0.0,1.0,0.0,6,10
1,mLspw4gJNTQGN,2004,Atlanta Falcons,299.0,422.0,-123.0,4357.0,930.0,4.7,31.0,...,0.392857,0.500000,0.030435,0.045652,1.019631,0.0,0.0,0.0,11,5
2,zyhFXj1nywrm1,2004,Baltimore Ravens,391.0,281.0,110.0,4929.0,1009.0,4.9,38.0,...,0.444015,0.522892,0.038554,0.045783,0.910664,0.0,0.0,0.0,9,7
3,3VqeLK0LS9rpw,2004,Buffalo Bills,243.0,279.0,-36.0,4348.0,980.0,4.4,34.0,...,0.358209,0.583665,0.021912,0.033865,0.895267,0.0,0.0,0.0,9,7
4,jJBGuksuDJF5L,2004,Carolina Panthers,325.0,304.0,21.0,5141.0,1008.0,5.1,31.0,...,0.401408,0.586957,0.041304,0.034783,1.274340,0.0,1.0,0.0,7,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,IwSI92ZDKoazn,2020,San Francisco 49ers,479.0,310.0,169.0,6097.0,1012.0,6.0,23.0,...,0.327381,0.692469,0.058577,0.027197,1.102205,0.0,0.0,0.0,6,10
540,4M8AK5p0WXs1N,2020,Seattle Seahawks,405.0,398.0,7.0,5991.0,1046.0,5.7,20.0,...,0.354839,0.659574,0.059961,0.011605,1.347171,0.0,1.0,1.0,12,4
541,QMAZH1v922YLu,2020,Tampa Bay Buccaneers,458.0,449.0,9.0,6366.0,1086.0,5.9,41.0,...,0.229462,0.606349,0.052381,0.047619,0.854898,0.0,0.0,1.0,11,5
542,ltMyRE3yvVUaw,2020,Tennessee Titans,402.0,331.0,71.0,5805.0,949.0,6.1,17.0,...,0.328076,0.662946,0.064732,0.017857,0.917394,0.0,0.0,0.0,11,5


In [28]:
df_seasons_total["win_percentage"]=df_seasons_total["total_wins"]/(df_seasons_total["total_wins"]+df_seasons_total["total_losses"])
df_seasons_total

,id,year,team,points_y_1,points_opp_y_1,points_diff_y_1,total_yards_y_1,plays_offense_y_1,yds_per_play_offense_y_1,turnovers_y_1,...,pass_cmp_per_y_1,pass_td_per_y_1,pass_int_per_y_1,prop_pyth_win_a_win_y_1,prop_pyth_win_a_win_me_7_y_1,prop_pyth_win_a_win_ma_12_y_1,margin_victory_m_1_y_1,total_wins,total_losses,win_percentage
0,GJizlkzQFaqGA,2004,Arizona Cardinals,225.0,452.0,-227.0,4490.0,981.0,4.6,36.0,...,0.567416,0.033708,0.041199,1.556007,0.0,1.0,0.0,6,10,0.3750
1,mLspw4gJNTQGN,2004,Atlanta Falcons,299.0,422.0,-123.0,4357.0,930.0,4.7,31.0,...,0.500000,0.030435,0.045652,1.019631,0.0,0.0,0.0,11,5,0.6875
2,zyhFXj1nywrm1,2004,Baltimore Ravens,391.0,281.0,110.0,4929.0,1009.0,4.9,38.0,...,0.522892,0.038554,0.045783,0.910664,0.0,0.0,0.0,9,7,0.5625
3,3VqeLK0LS9rpw,2004,Buffalo Bills,243.0,279.0,-36.0,4348.0,980.0,4.4,34.0,...,0.583665,0.021912,0.033865,0.895267,0.0,0.0,0.0,9,7,0.5625
4,jJBGuksuDJF5L,2004,Carolina Panthers,325.0,304.0,21.0,5141.0,1008.0,5.1,31.0,...,0.586957,0.041304,0.034783,1.274340,0.0,1.0,0.0,7,9,0.4375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,IwSI92ZDKoazn,2020,San Francisco 49ers,479.0,310.0,169.0,6097.0,1012.0,6.0,23.0,...,0.692469,0.058577,0.027197,1.102205,0.0,0.0,0.0,6,10,0.3750
540,4M8AK5p0WXs1N,2020,Seattle Seahawks,405.0,398.0,7.0,5991.0,1046.0,5.7,20.0,...,0.659574,0.059961,0.011605,1.347171,0.0,1.0,1.0,12,4,0.7500
541,QMAZH1v922YLu,2020,Tampa Bay Buccaneers,458.0,449.0,9.0,6366.0,1086.0,5.9,41.0,...,0.606349,0.052381,0.047619,0.854898,0.0,0.0,1.0,11,5,0.6875
542,ltMyRE3yvVUaw,2020,Tennessee Titans,402.0,331.0,71.0,5805.0,949.0,6.1,17.0,...,0.662946,0.064732,0.017857,0.917394,0.0,0.0,0.0,11,5,0.6875


In [30]:
# Guardamos la base en un csv
df_seasons_total.to_csv(r"/Users/damian/Documents/Maestria 2025-01/Aprendizaje maquina probabilistico/Proyecto intermedio/Base_proyecto.csv", index=False)